In [32]:
%load_ext autoreload
%autoreload 2
from building_dection import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Define the EPSG codes for the source and target coordinate systems
src_crs = 'EPSG:4326'
tgt_crs = 'EPSG:32610'  # UTM zone 10N in WGS84

labels = '/scratch/toml/sn7/qgis/labels2.geojson'
reproj_labels = '/scratch/toml/sn7/qgis/labels2_reproj.geojson'
raster_labels = '/scratch/toml/sn7/qgis/labels2.tif'
extent = [600000.0, 4190220.0, 709800.0, 4300020.0]

# Reproject the GeoJSON to the target coordinate system
reproject_geojson(labels, src_crs, tgt_crs, out_fn=reproj_labels)
rasterize_labels(reproj_labels, raster_labels, extent)


In [101]:

l1c = '/scratch/toml/sn7/qgis/l1c_set1 (copy).tif'
l1c_crop = '/scratch/toml/sn7/qgis/l1c_set1_crop.tif'
raster_labels = '/scratch/toml/sn7/qgis/labels2.tif'
raster_labels_crop = '/scratch/toml/sn7/qgis/labels2_crop.tif'
crop_to_extent(labels, l1c, l1c_crop)
crop_to_extent(labels, raster_labels, raster_labels_crop)


GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AXIS["Latitude",NORTH],
    AXIS["Longitude",EAST],
    AUTHORITY["EPSG","4326"]]
(-121.68466515096087, -121.64058492652846, 37.9615752659464, 37.99617689154867)
PROJCS["WGS 84 / UTM zone 10N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-123],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Eas